In [6]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

import numpy as np

In [3]:
def get_linear_entangelment_ansatz(num_of_qubits, thetas, input_state, circuit_depth=3):
    quantum_register = QuantumRegister(num_of_qubits, name="qubit")
    quantum_circuit = QuantumCircuit(quantum_register)
    quantum_circuit.initialize(input_state)

    for iteration in range(circuit_depth):
        for qubit_index in range(num_of_qubits):
            RY_theta_index = iteration*2*num_of_qubits + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])
            
        for qubit_index in range(num_of_qubits - 1):
            quantum_circuit.append(CXGate(), [quantum_register[qubit_index], quantum_register[qubit_index + 1]])

    for qubit_index in range(num_of_qubits):
            RY_theta_index = 2*num_of_qubits*circuit_depth + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])

    return quantum_circuit

In [4]:
def get_full_entangelment_ansatz(num_of_qubits, thetas, input_state, circuit_depth=3):
    quantum_register = QuantumRegister(num_of_qubits, name="qubit")
    quantum_circuit = QuantumCircuit(quantum_register)
    quantum_circuit.initialize(input_state)

    for iteration in range(circuit_depth):
        for qubit_index in range(num_of_qubits):
            RY_theta_index = iteration*2*num_of_qubits + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])
        for qubit_index in range(num_of_qubits - 1):
            for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                quantum_circuit.append(CXGate(), [quantum_register[qubit_index], quantum_register[target_qubit_index]])

    for qubit_index in range(num_of_qubits):
            RY_theta_index = 2*num_of_qubits*circuit_depth + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])

    return quantum_circuit

## Linear entangelment ansatz

In [5]:
initial_thetas = np.random.uniform(low=0, high=360, size=32)
initial_eigenvector = np.identity(16)[0]
print(get_linear_entangelment_ansatz(4, initial_thetas, 3))

         ┌────────────────┐┌────────────┐┌────────────┐     ┌────────────┐»
qubit_0: ┤0               ├┤ Ry(159.83) ├┤ Rz(25.965) ├──■──┤ Ry(110.46) ├»
         │                │├────────────┤├────────────┤┌─┴─┐└────────────┘»
qubit_1: ┤1               ├┤ Ry(60.518) ├┤ Rz(72.181) ├┤ X ├──────■───────»
         │  Initialize(3) │├────────────┤├────────────┤└───┘    ┌─┴─┐     »
qubit_2: ┤2               ├┤ Ry(356.05) ├┤ Rz(307.48) ├─────────┤ X ├─────»
         │                │├────────────┤├────────────┤         └───┘     »
qubit_3: ┤3               ├┤ Ry(130.91) ├┤ Rz(252.81) ├───────────────────»
         └────────────────┘└────────────┘└────────────┘                   »
«         ┌────────────┐                            ┌────────────┐»
«qubit_0: ┤ Rz(354.94) ├────────────────────■───────┤ Ry(241.64) ├»
«         ├────────────┤┌────────────┐    ┌─┴─┐     └────────────┘»
«qubit_1: ┤ Ry(333.54) ├┤ Rz(313.63) ├────┤ X ├───────────■───────»
«         └────────────┘├────────────┤┌───┴─

In [10]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=24)
initial_eigenvector = np.identity(8)[0]
print(get_linear_entangelment_ansatz(3, initial_thetas, 3))

         ┌────────────────┐┌────────────┐┌────────────┐     ┌────────────┐»
qubit_0: ┤0               ├┤ Ry(5.1497) ├┤ Rz(5.9969) ├──■──┤ Ry(5.0024) ├»
         │                │├────────────┤├────────────┤┌─┴─┐└────────────┘»
qubit_1: ┤1 Initialize(3) ├┤ Ry(1.4156) ├┤ Rz(2.8799) ├┤ X ├──────■───────»
         │                │├────────────┤├────────────┤└───┘    ┌─┴─┐     »
qubit_2: ┤2               ├┤ Ry(5.6369) ├┤ Rz(1.5474) ├─────────┤ X ├─────»
         └────────────────┘└────────────┘└────────────┘         └───┘     »
«         ┌────────────┐                   ┌─────────────┐┌────────────┐»
«qubit_0: ┤ Rz(6.2607) ├────────────────■──┤ Ry(0.66299) ├┤ Rz(2.2867) ├»
«         ├────────────┤┌────────────┐┌─┴─┐└─────────────┘├────────────┤»
«qubit_1: ┤ Ry(2.5677) ├┤ Rz(2.1184) ├┤ X ├───────■───────┤ Ry(5.7155) ├»
«         ├────────────┤├───────────┬┘└───┘     ┌─┴─┐     ├───────────┬┘»
«qubit_2: ┤ Ry(1.2866) ├┤ Rz(1.429) ├───────────┤ X ├─────┤ Ry(1.252) ├─»
«         └────────────┘

In [7]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=16)
initial_eigenvector = np.identity(4)[0]
print(get_linear_entangelment_ansatz(2, initial_thetas, 3))

         ┌────────────────┐┌─────────────┐┌────────────┐     ┌────────────┐»
qubit_0: ┤0               ├┤ Ry(0.83747) ├┤ Rz(1.4846) ├──■──┤ Ry(0.6268) ├»
         │  Initialize(3) │└┬───────────┬┘├────────────┤┌─┴─┐├───────────┬┘»
qubit_1: ┤1               ├─┤ Ry(4.375) ├─┤ Rz(1.9665) ├┤ X ├┤ Ry(1.614) ├─»
         └────────────────┘ └───────────┘ └────────────┘└───┘└───────────┘ »
«         ┌────────────┐      ┌────────────┐┌────────────┐     ┌────────────┐»
«qubit_0: ┤ Rz(5.3903) ├──■───┤ Ry(5.4686) ├┤ Rz(1.1114) ├──■──┤ Ry(4.5434) ├»
«         ├────────────┤┌─┴─┐┌┴────────────┤├───────────┬┘┌─┴─┐├────────────┤»
«qubit_1: ┤ Rz(4.4936) ├┤ X ├┤ Ry(0.23003) ├┤ Rz(2.147) ├─┤ X ├┤ Ry(6.0521) ├»
«         └────────────┘└───┘└─────────────┘└───────────┘ └───┘└────────────┘»
«         ┌──────────────┐
«qubit_0: ┤ Rz(0.054925) ├
«         └┬────────────┬┘
«qubit_1: ─┤ Rz(2.1577) ├─
«          └────────────┘ 


## Full entangelment ansatz

In [11]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=32)
initial_eigenvector = np.identity(16)[0]
print(get_full_entangelment_ansatz(4, initial_thetas, 3))

         ┌────────────────┐┌────────────┐┌─────────────┐               »
qubit_0: ┤0               ├┤ Ry(4.6619) ├┤ Rz(0.41515) ├──■────■────■──»
         │                │├────────────┤└┬────────────┤┌─┴─┐  │    │  »
qubit_1: ┤1               ├┤ Ry(4.8225) ├─┤ Rz(5.0193) ├┤ X ├──┼────┼──»
         │  Initialize(3) │├────────────┤ ├────────────┤└───┘┌─┴─┐  │  »
qubit_2: ┤2               ├┤ Ry(2.9944) ├─┤ Rz(3.6326) ├─────┤ X ├──┼──»
         │                │├────────────┤ ├───────────┬┘     └───┘┌─┴─┐»
qubit_3: ┤3               ├┤ Ry(3.5793) ├─┤ Rz(1.981) ├───────────┤ X ├»
         └────────────────┘└────────────┘ └───────────┘           └───┘»
«         ┌───────────┐┌────────────┐                                          »
«qubit_0: ┤ Ry(4.902) ├┤ Rz(4.5362) ├──────────────────────────────────■───────»
«         └───────────┘└────────────┘┌────────────┐┌────────────┐    ┌─┴─┐     »
«qubit_1: ──────■────────────■───────┤ Ry(5.3632) ├┤ Rz(5.8175) ├────┤ X ├─────»
«             ┌─┴─┐

In [11]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=24)
initial_eigenvector = np.identity(8)[0]
print(get_full_entangelment_ansatz(3, initial_thetas, 3))

         ┌────────────────┐┌───────────┐ ┌────────────┐          ┌────────────┐»
qubit_0: ┤0               ├┤ Ry(3.846) ├─┤ Rz(1.2031) ├──■────■──┤ Ry(2.4913) ├»
         │                │├───────────┴┐├────────────┤┌─┴─┐  │  └────────────┘»
qubit_1: ┤1 Initialize(3) ├┤ Ry(2.9839) ├┤ Rz(3.2826) ├┤ X ├──┼────────■───────»
         │                │├────────────┤├───────────┬┘└───┘┌─┴─┐    ┌─┴─┐     »
qubit_2: ┤2               ├┤ Ry(1.2838) ├┤ Rz(3.399) ├──────┤ X ├────┤ X ├─────»
         └────────────────┘└────────────┘└───────────┘      └───┘    └───┘     »
«          ┌────────────┐                        ┌────────────┐┌────────────┐»
«qubit_0: ─┤ Rz(5.1594) ├────────────────■────■──┤ Ry(4.5917) ├┤ Rz(2.3964) ├»
«         ┌┴────────────┤┌────────────┐┌─┴─┐  │  └────────────┘├────────────┤»
«qubit_1: ┤ Ry(0.81103) ├┤ Rz(4.8218) ├┤ X ├──┼────────■───────┤ Ry(1.5008) ├»
«         ├─────────────┤├────────────┤└───┘┌─┴─┐    ┌─┴─┐     ├────────────┤»
«qubit_2: ┤ Ry(0.09039) ├┤ Rz(2.1141) 

In [8]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=16)
initial_eigenvector = np.identity(4)[0]
print(get_full_entangelment_ansatz(2, initial_thetas, 3))

         ┌────────────────┐ ┌────────────┐┌───────────┐      ┌────────────┐»
qubit_0: ┤0               ├─┤ Ry(3.5619) ├┤ Rz(3.688) ├───■──┤ Ry(3.7257) ├»
         │  Initialize(3) │┌┴────────────┤├───────────┴┐┌─┴─┐├────────────┤»
qubit_1: ┤1               ├┤ Ry(0.34152) ├┤ Rz(4.8209) ├┤ X ├┤ Ry(4.4549) ├»
         └────────────────┘└─────────────┘└────────────┘└───┘└────────────┘»
«         ┌────────────┐     ┌────────────┐┌─────────────┐     ┌────────────┐»
«qubit_0: ┤ Rz(2.0927) ├──■──┤ Ry(5.4044) ├┤ Rz(0.95238) ├──■──┤ Ry(5.6428) ├»
«         ├────────────┤┌─┴─┐├────────────┤└┬────────────┤┌─┴─┐├────────────┤»
«qubit_1: ┤ Rz(1.6194) ├┤ X ├┤ Ry(3.6951) ├─┤ Rz(1.1115) ├┤ X ├┤ Ry(1.8133) ├»
«         └────────────┘└───┘└────────────┘ └────────────┘└───┘└────────────┘»
«         ┌─────────────┐ 
«qubit_0: ┤ Rz(0.51745) ├─
«         ├─────────────┴┐
«qubit_1: ┤ Rz(0.045128) ├
«         └──────────────┘
